In [1]:
from squad_goals import Agent, Task, Plan, Workflow
from squad_goals.tools import SerpTool
from squad_goals.llms.openai import OpenAILLM
from squad_goals.llms.groq import GroqLLM
from squad_goals.tools import SerpTool
import os

# Set API keys as environment variables
os.environ['SERP_API_KEY'] = 'your_openai_key'
os.environ['GROQ_API_KEY'] = 'your_groq_key'

os.environ['OPENAI_API_KEY'] = 'your_serp_key'

In [8]:
# Initialize the OpenAI LLM
openai_llm = OpenAILLM(model_name='gpt-4o')
groq_llm = GroqLLM(model_name='gemma2-9b-it')

In [9]:
%timeit openai_llm.generate([{'role': 'user', 'content': 'Tell me about Sinan Ozdemir.'}])

2.67 s ± 453 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit groq_llm.generate([{'role': 'user', 'content': 'Tell me about Sinan Ozdemir.'}])

260 ms ± 75.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# Initialize the SerpTool for web searching
serp_tool = SerpTool()

In [12]:
# Create an agent with tools and an LLM
agent_big = Agent(
    tools=[serp_tool],
    llm=openai_llm,
    verbose=False
)
agent_small = Agent(
    tools=[],
    llm=groq_llm,
    verbose=False
)

In [36]:
goal = 'Tell me about Sinan Ozdemir.'
steps = [
    'Make at least 2 web lookups about Sinan Ozdemir.',
    'Summarize the information you found about Sinan Ozdemir in a few sentences in Turkish.'
]

In [37]:
plan = Plan(
    goal=goal, 
    steps=steps
)
workflow = Workflow(
    plan=plan,
    agents=[agent_big, agent_small],  # our squad!
    goal=goal
)
for event in workflow.run(yield_events=True):
    print(f'Event: {event["event"]} | Tool: {event.get("tool", None)}')

Event: next_step | Tool: None
Event: next_agent_action | Tool: Return Final Answer Tool
Event: tool_selected | Tool: Return Final Answer Tool
Event: tool_run | Tool: Return Final Answer Tool
Event: tool_result | Tool: Return Final Answer Tool
Event: agent_completed | Tool: None
Event: step_result | Tool: None
Event: next_step | Tool: None
Event: next_agent_action | Tool: Return Final Answer Tool
Event: tool_selected | Tool: Return Final Answer Tool
Event: tool_run | Tool: Return Final Answer Tool
Event: tool_result | Tool: Return Final Answer Tool
Event: agent_completed | Tool: None
Event: step_result | Tool: None
Event: workflow_complete | Tool: None


In [38]:
event

{'event': 'workflow_complete',
 'results': ['"Sinan Ozdemir is an AI expert, consultant, author, and founder of LoopGenius. He has written several books, including the \'Quick Start Guide to LLMs,\' and has held teaching positions, notably at Johns Hopkins University. He is recognized for his expertise in Generative AI and NLP and has made contributions to healthcare data analytics."',
  '"Sinan Ozdemir, yapay zeka uzman\\u0131, dan\\u0131\\u015fman, yazar ve LoopGenius\'in kurucusudur. \'H\\u0131zl\\u0131 Ba\\u015flang\\u0131\\u00e7 Rehberi LLMs\' dahil olmak \\u00fczere birka\\u00e7 kitap yazm\\u0131\\u015ft\\u0131r ve Johns Hopkins \\u00dcniversitesi\'nde ders vermi\\u015f. Generatif yapay zeka ve NLP alanlar\\u0131nda uzmanl\\u0131\\u011f\\u0131 ile bilinir ve sa\\u011fl\\u0131k veri analizi alan\\u0131nda katk\\u0131larda bulunmu\\u015ftur."']}

In [49]:
import ast

for step, result in zip(steps, plan.results):
    print('### Step ###')
    print(step)
    print('### Result ###')
    try:
        print(ast.literal_eval(result))  # to print Turkish characters correctly
    except (SyntaxError, ValueError):
        print(result)
    print()


### Step ###
Make at least 2 web lookups about Sinan Ozdemir.
### Result ###
Sinan Ozdemir is an AI expert, consultant, author, and founder of LoopGenius. He has written several books, including the 'Quick Start Guide to LLMs,' and has held teaching positions, notably at Johns Hopkins University. He is recognized for his expertise in Generative AI and NLP and has made contributions to healthcare data analytics.

### Step ###
Summarize the information you found about Sinan Ozdemir in a few sentences in Turkish.
### Result ###
Sinan Ozdemir, yapay zeka uzmanı, danışman, yazar ve LoopGenius'in kurucusudur. 'Hızlı Başlangıç Rehberi LLMs' dahil olmak üzere birkaç kitap yazmıştır ve Johns Hopkins Üniversitesi'nde ders vermiş. Generatif yapay zeka ve NLP alanlarında uzmanlığı ile bilinir ve sağlık veri analizi alanında katkılarda bulunmuştur.
